In [ ]:
import yfinance as yf
import pandas as pd

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
company_names = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "GOOGL": "Alphabet Inc.",
    "AMZN": "Amazon.com Inc.",
    "TSLA": "Tesla Inc."
}

data = yf.download(tickers, period="2y", group_by="ticker")

all_data = []
id_counter = 0

for ticker in tickers:
    df = data[ticker].copy()
    df["Ticker"] = ticker
    df["Company Name"] = company_names[ticker]
    df["Index"] = range(id_counter, id_counter + len(df))
    id_counter += len(df)
    df["Datetime"] = df.index
    all_data.append(df)

final_data = pd.concat(all_data).reset_index(drop=True)



[*********************100%***********************]  5 of 5 completed


Price,Open,High,Low,Close,Volume,Ticker,Company Name,ID,Datetime
0,136.682651,141.828549,136.464939,139.641541,81760300,AAPL,Apple Inc.,0,2023-01-23
1,138.849854,141.670202,138.839964,141.046753,66435100,AAPL,Apple Inc.,1,2023-01-24
2,139.423806,140.947773,137.365450,140.383713,65799300,AAPL,Apple Inc.,2,2023-01-25
3,141.680111,142.748874,140.423323,142.461899,54105100,AAPL,Apple Inc.,3,2023-01-26
4,141.670184,145.697821,141.591015,144.411346,70555800,AAPL,Apple Inc.,4,2023-01-27


In [95]:
import pandas as pd
import numpy as np
import yfinance as yf
from xgboost import XGBRegressor

# List of tickers and their corresponding company names
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
company_names = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "GOOGL": "Alphabet Inc.",
    "AMZN": "Amazon.com Inc.",
    "TSLA": "Tesla Inc."
}

# Download stock data from Yahoo Finance
data = yf.download(tickers, period="1y", group_by="ticker")

# Prepare data storage
all_data = []

# Iterate through each stock ticker
for ticker in tickers:
    # Prepare the dataframe for the stock
    df = data[ticker].copy()
    df["Ticker"] = ticker
    df["Company Name"] = company_names[ticker]
    df["Datetime"] = df.index

    # Add a rolling window of 30 days for feature engineering (previous 30 days)
    look_back = 30

    # Create a model for each stock and fit it on the historical data
    model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
    for i in range(look_back, len(df)):
        # Extract rolling window data
        X_train = df['Close'].iloc[i - look_back:i].values.reshape(1, -1)
        y_train = df['Close'].iloc[i]
        model.fit(X_train, [y_train])

    # Predict the next day's price
    last_window = df['Close'].iloc[-look_back:].values.reshape(1, -1)  # Use the last 30 days as input
    next_day_prediction = model.predict(last_window)[0]

    # Prepare a DataFrame for the next day's prediction
    next_day_date = pd.date_range(df.index[-1] + pd.Timedelta(days=1), periods=1, freq='B')  # Next business day
    next_day_df = pd.DataFrame({
        'predicted_close': [next_day_prediction],
        'Ticker': ticker,
        'Company Name': company_names[ticker],
        'Datetime': next_day_date
    })

    # Append the next day's prediction to the main dataframe
    all_data.append(next_day_df)

# Concatenate all predictions for all tickers into a single dataframe
final_predictions = pd.concat(all_data)

# Display the final predictions
final_predictions

[*********************100%***********************]  5 of 5 completed


,predicted_close,Ticker,Company Name,Datetime
0,222.550293,AAPL,Apple Inc.,2025-01-23
0,445.130005,MSFT,Microsoft Corporation,2025-01-23
0,198.380005,GOOGL,Alphabet Inc.,2025-01-23
0,233.192001,AMZN,Amazon.com Inc.,2025-01-23
0,419.850006,TSLA,Tesla Inc.,2025-01-23


In [100]:
import wikipedia
import pandas as pd

# Ticker listesi ve şirket isimleri
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
company_names = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "GOOGL": "Alphabet Inc.",
    "AMZN": "Amazon.com Inc.",
    "TSLA": "Tesla Inc."
}

# Şirket bilgilerini saklamak için bir liste
data = []

# Şirketler için özet bilgileri al
for ticker in tickers:
    original_name = company_names[ticker]
    company_name = original_name  # Orijinal isim
    
    try:
        # Wikipedia'dan özet bilgileri al
        summary = wikipedia.summary(company_name, sentences=2)  # İlk 2 cümleyi çek
    except wikipedia.exceptions.PageError:
        # Sayfa bulunamadıysa "Inc.", "Corporation", ".com" gibi ifadeleri kaldır
        cleaned_name = company_name.replace("Inc.", "").replace("Corporation", "").replace(".com", "").strip()
        try:
            summary = wikipedia.summary(cleaned_name, sentences=2)  # Temiz isimle tekrar dene
            company_name = cleaned_name  # Başarılı olursa bu ismi kullan
        except wikipedia.exceptions.PageError:
            summary = "Sayfa bulunamadı."  # Hala bulunamazsa mesaj yaz
        except wikipedia.exceptions.DisambiguationError as e:
            summary = f"Çok fazla seçenek bulundu: {e.options[:5]}..."
    except wikipedia.exceptions.DisambiguationError as e:
        summary = f"Çok fazla seçenek bulundu: {e.options[:5]}..."
    except Exception as e:
        summary = f"Hata: {str(e)}"  # Diğer hatalar
    
    # Veriyi listeye ekle
    data.append({"Ticker": ticker, "Original Company Name": original_name, "Final Company Name": company_name, "Summary": summary})

# Veriyi Pandas DataFrame'e dönüştür
df = pd.DataFrame(data)

# DataFrame'i ekrana yazdır
df.head()


,Ticker,Original Company Name,Final Company Name,Summary
0,AAPL,Apple Inc.,Apple,"In mathematics and computer science, apply is ..."
1,MSFT,Microsoft Corporation,Microsoft Corporation,Microsoft Corporation is an American multinati...
2,GOOGL,Alphabet Inc.,Alphabet Inc.,Alphabet Inc. is an American multinational tec...
3,AMZN,Amazon.com Inc.,Amazon.com Inc.,"Amazon.com, Inc., doing business as Amazon (, ..."
4,TSLA,Tesla Inc.,Tesla Inc.,"Tesla, Inc. ( TESS-lə or TEZ-lə) is an Americ..."
